In [1]:
# LangChain은 언어 모델(LLM)을 활용해 다양한 어플리케이션을 개발할 수 있는 프레임워크
!pip install -U langchain langchain-core langchain-community langchain-openai
!pip install -U langchain-splitters openai tiktoken
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully 

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
# Colab 환경에서 .env 파일을 사용하려면, 왼쪽 파일 탐색기에서 새 파일을 생성하고 '.env'로 이름을 지정한 후,
# OPENAI_API_KEY='여기에_실제_OpenAI_API_키를_입력하세요' 와 같이 내용을 입력해야 합니다.
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# 만약 .env 파일 사용이 어렵다면, 아래 주석을 해제하고 직접 API 키를 할당하세요.
api_key = "키값입력!!"

if not api_key:
  raise RuntimeError(".env 파일에 OPENAI_API_KEY가 없거나 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")

print("API 키가 성공적으로 로드되었습니다.")

API 키가 성공적으로 로드되었습니다.


In [9]:
from re import search
# LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=api_key)

#문서 로딩
docs = TextLoader('sample.txt', encoding='utf-8').load()
print(f'문서 갯수:{len(docs)}')
print(docs)
print(docs[0].page_content)

print()
#문서 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = splitter.split_documents(docs)
print(f'생성된 청크 수 : {len(chunks)}')
print(chunks)
print(chunks[0].page_content)

문서 갯수:1
[Document(metadata={'source': 'sample.txt'}, page_content='이데일리 김윤지 기자] 미국 하원이 12일(현지시간) 상원이 통과시킨 셧다운(일시적 업무 중지) 종료를 위한 합의안을 최종 가결했다. 임시예산안이 하원을 최종 통과하면서 도널드 트럼프 대통령의 서명만 남았다.\n\n미 하원은 이날 오후 전체회의에서 단기 지출법안(CR·임시예산안)에 대한 표결을 실시해 이를 통과시켰다.\n백악관은 앞서 트럼프 대통령이 해당 임시예산안을 지지한다면서 이날 오후 9시 45분(한국시간 13일 오전 11시 45분) 서명식을 진행한다고 밝혔다.\n\n표결에 앞서 민주당 하원 원내대표 하킴 제프리스(뉴욕)는 “이 싸움이 끝나는 방식은 단 두 가지뿐이다. 올해 공화당이 마침내 ‘오바마케어‘ 세액공제를 연장하기로 결정하든지, 아니면 미국 국민이 내년에 공화당을 그들의 자리에서 몰아내고 도널드 트럼프의 대통령직을 영원히 끝내든지다”라고 말했다.\n\n이어 마이크 존슨(공화·루이지애나) 하원의장은 “이 모든 과정은 완전히 무의미했다. 잘못된 것이었고, 잔혹했다“며 ”우리는 더 이상 미국인들을 위해 시간을 낭비할 수 없다“고 말했다.')]
이데일리 김윤지 기자] 미국 하원이 12일(현지시간) 상원이 통과시킨 셧다운(일시적 업무 중지) 종료를 위한 합의안을 최종 가결했다. 임시예산안이 하원을 최종 통과하면서 도널드 트럼프 대통령의 서명만 남았다.

미 하원은 이날 오후 전체회의에서 단기 지출법안(CR·임시예산안)에 대한 표결을 실시해 이를 통과시켰다.
백악관은 앞서 트럼프 대통령이 해당 임시예산안을 지지한다면서 이날 오후 9시 45분(한국시간 13일 오전 11시 45분) 서명식을 진행한다고 밝혔다.

표결에 앞서 민주당 하원 원내대표 하킴 제프리스(뉴욕)는 “이 싸움이 끝나는 방식은 단 두 가지뿐이다. 올해 공화당이 마침내 ‘오바마케어‘ 세액공제를 연장하기로 결정하든지, 아니면 미국 국민이 내년에 공화당을 그들의 자리에서 몰아내고 

In [10]:
print()
# 임베딩 & 벡터 스토어
# embeddings = OpenAIEmbeddings()   # 유료
# vectorstore = Chroma.from_documents(chunks, embeddings)
# retriever = vectorstore.as_retriever(search_kwargs={'k':3})
# print(retriever)

def format_docs(docs):
  return '\n\n'.join(d.page_content for d in docs)

from langchain_core.runnables import RunnableLambda
docs_runnable = RunnableLambda(lambda _: chunks)  # 고정 청크를 Runnable로 래핑

prompt = ChatPromptTemplate.from_template(
    '답하세요\n'
    '{context}\n'
    '질문:{question}'
)

# 고정된 데이터는 RunnableLambda로 감싸야 한다.
chain =(
    {   # 1) 입력 데이터 구성
        # context 생성 규칙
        # docs_runnable: 실행될 때 항상 'chunks' 리스트를 반환하는 RunnableLambda
        # docs_runnable | RunnableLambda(format_docs)
        #      : chunks 리스트를 문서 문자열(context)로 변환
        'context':docs_runnable | RunnableLambda(format_docs),
        # question 생성 규칙 : 사용자 입력(question)을 그대로 다음 단계로 전달
        'question':RunnablePassthrough()
    }
    # 2) PromptTemplate 적용 : {context, question}을 prompt 템플릿에 채워 넣음
    | prompt
    # 3) 템플릿이 완성되면 llm을 호출해 답변 생성
    | llm
    # 4) LLM 결과에서 message 객체 대신 문자열만 추출하여 텍스트로 출력
    | StrOutputParser()
)

user_question = "현대자동차에 대해 설명해"
answer = chain.invoke(user_question)
print(f'질문 : {user_question}')
print(f'대답 : {answer}')


# 수행 순서 ----------------------------------
# 사용자 질문 ("현대자동차 설명해")
#         │
#         ▼
#  RunnablePassthrough()  → question 유지
#         │
#  docs_runnable → chunks 꺼내기
#         │
#  format_docs → context 문자열 생성
#         │
#         └───> {"context": ..., "question": ...}
#                     │
#                     ▼
#                 PromptTemplate
#                     │
#                     ▼
#                     LLM
#                     │
#                     ▼
#             StrOutputParser()
#                     │
#                     ▼
#                 최종 answer



질문 : 현대자동차에 대해 설명해
대답 : 현대자동차는 대한민국의 대표적인 자동차 제조업체로, 1967년에 설립되었습니다. 본사는 서울특별시에 위치하고 있으며, 글로벌 시장에서 중요한 위치를 차지하고 있습니다. 현대자동차는 승용차, SUV, 상용차 등 다양한 모델을 생산하며, 전 세계 여러 나라에 판매하고 있습니다.

회사는 기술 혁신과 품질 향상에 집중하여 세계적인 경쟁력을 갖추고 있으며, 친환경 차량 개발에도 많은 노력을 기울이고 있습니다. 특히 전기차와 수소전지차 분야에서의 연구 개발이 두드러집니다. 현대자동차는 '모빌리티 솔루션'을 제공하는 기업으로서의 비전을 가지고 있으며, 자율주행, 커넥티드 카 등 미래 자동차 기술에도 적극적으로 투자하고 있습니다.

또한, 현대자동차는 글로벌 생산 네트워크를 갖추고 있으며, 미국, 유럽, 아시아 등 여러 지역에 생산 공장을 운영하고 있습니다. 이는 다양한 시장의 수요에 대응하고, 효율적인 공급망 관리를 가능하게 합니다. 현대자동차는 품질, 디자인, 성능에서 높은 평가를 받고 있으며, 전 세계적으로 많은 팬층을 보유하고 있습니다.
